In [1]:
using Revise
using LinearAlgebra
using AdaptiveTransportMap
using Statistics
using Distributions
using Test
using ForwardDiff
using SpecialFunctions

┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1278
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116


In [2]:
Nx = 3
Ne = 100

X = randn(Nx, Ne) .* randn(Nx, Ne)

3×100 Array{Float64,2}:
 -0.506034  -0.842495  -0.399615    …  -0.0115519  -0.0753821   1.41811
  1.49441    0.773311  -0.00325653      0.198769    0.24023    -0.280636
 -0.467444   0.185993  -0.942934       -1.13423     0.18967     0.0723793

In [62]:
p = 2
order = -1*ones(Int64, Nx)
order[end] = p

λ = 0.1
δ = 0.0
γ = 2.0

C = SparseRadialMapComponent(Nx, order)

center_std(C, X; γ = γ)

In [63]:
xdiag = optimize(C, X, λ, δ)
@show xdiag
modify_a(xdiag, C)

xdiag = [-0.6605819724481781, 1.0570705611049904, 0.0, 2.0289165153288455, 0.31270986137743895]


In [64]:
C.ξ[1] = zeros(2)

2-element Array{Float64,1}:
 0.0
 0.0

In [80]:
X = randn(3,4)

3×4 Array{Float64,2}:
  0.473474  2.09537   -1.62193     1.20144
 -0.570037  0.431455   0.0729448   0.604184
 -0.397887  1.19417   -1.13854    -0.445431

In [81]:
sort(X; dims = 2)

3×4 Array{Float64,2}:
 -1.62193    0.473474    1.20144   2.09537
 -0.570037   0.0729448   0.431455  0.604184
 -1.13854   -0.445431   -0.397887  1.19417

In [67]:
fieldnames(typeof(C))

(:Nx, :p, :activedim, :ξ, :σ, :a)

In [79]:
    @test isapprox(quantile([0.5377;    1.8339;
             -2.2588;    0.8622;    0.3188;
                -1.3077;   -0.4336;    0.3426;
                    3.5784;    2.7694], collect(1:11)./(11+1.0); sorted=false, alpha = 0.5, beta = 0.5),
[-1.9417666666666666;
     -1.162016666666667;
     -0.4336 ;
      0.19339999999999974;
      0.33466666666666667;
      0.44015   ;
      0.6458666666666668 ;
      1.0241499999999994;
      1.8339    ;
      2.6134833333333343 ;
      3.308733333333333], atol = 1e-10)

Test Passed

In [77]:
quantile(sort([0.5377;    1.8339; -2.2588;    0.8622;    0.3188; -1.3077;   -0.4336;    0.3426; 3.5784;    2.7694]),[0.25; 0.5; 0.75], alpha = 0.5, beta = 0.5)

3-element Array{Float64,1}:
 -0.4336
  0.44015
  1.8339

In [70]:
?update_component

search: update_component



No documentation found.

`AdaptiveTransportMap.update_component` is a `Function`.

```
# 1 method for generic function "update_component":
[1] update_component(C::HermiteMapComponent, X, reduced_margin::Array{Int64,2}, S::Storage) in AdaptiveTransportMap at /media/mat/HDD/AdaptiveTransportMap/src/hermitemap/greedyfit.jl:431
```


In [49]:
# Create a radial map with order p for all the entries
Cfull = SparseRadialMapComponent(Nx, p)

# Compute centers and widths
center_std(Cfull, X; γ = 2.0)

In [50]:
### Evaluate the different basis

# Create weights
ψ_off, ψ_diag, ψ_∂k = compute_weights(Cfull, X)

([-0.5060342458307376 -0.8424945488393228 … -0.07538212662610277 1.4181088544263758; 0.4112214409429559 0.33048923092775484 … 0.44305093921690836 0.0962882457735615; … ; 0.10370205621754343 0.23530690982724844 … 0.32767175464878906 0.3614318285557624; 0.2016956834808597 0.33008398697933805 … 0.3610054535663662 0.31447779362939793], [-0.36157931795379117 -0.14640600356897468 … -0.1455680059360183 -0.17416966950579016; 0.2946009402268013 0.6477711017052892 … 0.6496895538799493 0.5868578621532539; 0.1644837638063545 0.5793349641684002 … 0.5819215071092174 0.49826559323642294; 0.04614178214323719 0.19418001696778922 … 0.19550473400300797 0.15632559419759445], [0.43936725566221746 0.2284248218142771 … 0.22741940931720678 0.260694865979447; 0.48511639289511516 0.5223025802927997 … 0.5212718390374709 0.5479048298243197; 0.4459891427684479 0.7039695167821562 … 0.7030218988758633 0.7182115138677226; 0.11843668943772773 0.3594356754693354 … 0.36116835068998754 0.3074677069363226])

In [51]:
C.p

3-element Array{Int64,1}:
 2
 2
 2

In [53]:
C.a[2]

3-element Array{Float64,1}:
  0.01998042858133154
  2.5365343994175134
 -4.353719767465045

In [27]:
xoff = zeros(6)

6-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [28]:
xdiag

5-element Array{Float64,1}:
 -0.6605819724481781
  1.0570705611049904
  0.0
  2.0289165153288455
  0.31270986137743895

In [29]:
L(x) = (1/(2*Ne))*norm((ψ_off'*x) + ψ_diag'*xdiag[2:end] .+ xdiag[1])^2 + λ*x'*x

L (generic function with 1 method)

In [30]:
ForwardDiff.gradient(x->L(x), xoff)

6-element Array{Float64,1}:
  0.11149954905702167
 -0.0017661664202742843
  0.0074999993566993665
  0.15108410694750185
  0.009462311261688443
  0.02182530671298463

In [32]:
b = vcat(ones(1,Ne), ψ_diag)'*xdiag;
J = 1/Ne*(ψ_off)*ψ_off'*xoff + (1/(2*Ne))*ψ_off*b + ((1/(2*Ne))*b'*ψ_off')' + λ*xoff           

6-element Array{Float64,1}:
  0.11149954905702171
 -0.0017661664202742785
  0.0074999993566993595
  0.15108410694750185
  0.009462311261688441
  0.021825306712984625

In [33]:
norm(J[1:3])

0.1117654632437668

In [34]:
norm(J[4:6])

0.1529453716843145

In [ ]:
@testset "Verify updateqrfactUnblocked! for a vector" begin

    ψ = randn(100, 20)
    ϕ = randn(100)

    F = qrfactUnblocked(ψ)
    Fold = deepcopy(F)
    G = qrfactUnblocked(hcat(ψ, ϕ))
    F = updateqrfactUnblocked!(F, ϕ)
    @test norm(F.τ-G.τ)<1e-14
    @test norm(F.factors - G.factors)<1e-14
end

@testset "Verify updateqrfactUnblocked! for a matrix" begin

    ψ = randn(100, 20)
    ϕ = randn(100, 5)

    F = qrfactUnblocked(ψ)
    Fold = deepcopy(F)
    G = qrfactUnblocked(hcat(ψ, ϕ))
    F = updateqrfactUnblocked!(F, ϕ)
    @test norm(F.τ-G.τ)<1e-14
    @test norm(F.factors - G.factors)<1e-14
end